17.2 Relational Databases and Structured Query Language

In [1]:
# Working with a Database in Python

import sqlite3

connection = sqlite3.connect('books.db')

In [2]:
# View the author's table contents

import pandas as pd

pd.options.display.max_columns = 10    # to set max number of columns

pd.read_sql('SELECT * FROM authors', connection,
            index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [3]:
# View the title's table contents

pd.read_sql('SELECT * FROM titles', connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016


In [4]:
# View author_ISBN table (first five rows)

df = pd.read_sql('SELECT * FROM author_ISBN', connection)

df.head()

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673


17.2.2 SELECT queries

In [5]:
# Retrieve only the columns first and last from the authors table

pd.read_sql('SELECT first, last FROM authors', connection)

,first,last
0,Paul,Deitel
1,Harvey,Deitel
2,Abbey,Deitel
3,Dan,Quirk
4,Alexander,Wald


17.2.3 WHERE Clause

In [6]:
# Select the title, edition, and copyright for all books with copyright years greater than 2016.
# String values in SQL queries are delimited by single (') quotes, as in '2016'

pd.read_sql("""SELECT title, edition, copyright
               FROM titles
               WHERE copyright > '2016'""", connection)

,title,edition,copyright
0,Intro to Python for CS and DS,1,2020
1,Java How to Program,11,2018
2,Visual C# How to Program,6,2017
3,C++ How to Program,10,2017
4,Android How to Program,3,2017


In [7]:
# Locate all authors whose last name starts with the letter D:

pd.read_sql("""SELECT id, first, last
                FROM authors
                WHERE last LIKE 'D%'""",
            connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel


In [8]:
# Select the rows of all the authors whose last names start with any character, followed by the
# letter b, followed by any number of additional characters (specified by %)

pd.read_sql("""SELECT id, first last
               FROM authors
               WHERE first LIKE '_b%'""",
            connection, index_col=['id'])

,last
id,
3,Abbey


17.2.4 ORDER BY Clause

In [9]:
# ORDER BY clause sorts a query's results into ascending or descending order, specified
# with ASC and DESC, respectively

#Sort the titles in ascending order:

pd.read_sql('SELECT title FROM titles ORDER BY title ASC',
            connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


In [10]:
# Sort the authors' names by last name, then by first name for any authors who have the same last name

pd.read_sql("""SELECT id, first, last
               FROM authors
               ORDER BY last, first""",
            connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel
4,Dan,Quirk
5,Alexander,Wald


In [11]:
# Sort the authors in descending order by last name and ascending order by first name for any
# authors who have the same last name:

pd.read_sql("""SELECT id, first, last
               FROM authors
               ORDER BY last DESC, first ASC""",
            connection, index_col=['id'])

,first,last
id,,
5,Alexander,Wald
4,Dan,Quirk
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel


In [12]:
# Combining the WHERE and ORDER BY Clauses

pd.read_sql("""SELECT isbn, title, edition, copyright
               FROM titles
               WHERE title LIKE '%How to Program'
               ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0133976890,C How to Program,8,2016
2,0134448235,C++ How to Program,10,2017
3,0132151006,Internet & WWW How to Program,5,2012
4,0134743350,Java How to Program,11,2018
5,0133406954,Visual Basic 2012 How to Program,6,2014
6,0134601548,Visual C# How to Program,6,2017
7,0136151574,Visual C++ How to Program,2,2008


17.2.5 INNER JOIN

In [13]:
# Merging data from multiple tables

pd.read_sql("""SELECT first, last, isbn
               FROM authors
               INNER JOIN author_ISBN
                   ON authors.id = author_ISBN.id
               ORDER BY last, first""", connection).head()

,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006


17.2.6 INSERT INTO Statement

In [14]:
# Use the sqlite3 Cursor object to execute SQL statements that modify the database

cursor = connection.cursor()

In [15]:
# Insert a new author named Sue Red into the authors table

cursor = cursor.execute("""INSERT INTO authors (first, last)
                           VALUES ('Sue', 'Red')""")

In [16]:
# View and confirm change

pd.read_sql('SELECT id, first, last FROM authors',
            connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Red


17.2.7 UPDATE Statement

In [17]:
# Update modifies existing values

# Update Sue Red's last name to 'Black'

cursor = cursor.execute("""UPDATE authors SET last='Black'
                           WHERE last='Red' AND first='Sue'""")

In [18]:
# Shows number of rows updated

cursor.rowcount

1

In [19]:
# Confirms the update

pd.read_sql('SELECT id, first, last FROM authors',
            connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Black


17.2.8 DELETE FROM Statement

In [20]:
# DELETE FROM statement removes rows from a table

# Removing Sue Black from the authors table using her author ID:

cursor = cursor.execute('DELETE FROM authors WHERE id=6')

cursor.rowcount    # checks number of rows deleted

1

In [21]:
# Confirms the update

pd.read_sql('SELECT id, first, last FROM authors',
            connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


17.2 Self Check

In [22]:
# 1) Select from the titles table all the titles and their edition numbers in descending order by
# edition number. Show only the first three results

pd.read_sql("""SELECT title, edition FROM titles
               ORDER BY edition DESC""", connection).head(3)

,title,edition
0,Java How to Program,11
1,C++ How to Program,10
2,C How to Program,8


In [24]:
# 2) Select from the authors table all authors whose first names start with 'A':

pd.read_sql("""SELECT * FROM authors
            WHERE first LIKE 'A%'""", connection)

,id,first,last
0,3,Abbey,Deitel
1,5,Alexander,Wald


In [25]:
# 3) Select from the titles table all titles that do not end with 'How to Program':

pd.read_sql("""SELECT isbn, title, edition, copyright
               FROM titles
               WHERE title NOT LIKE '%How to Program'
               ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134289366,Android 6 for Programmers,3,2016
1,0135404673,Intro to Python for CS and DS,1,2020
